# Factor Analysis & Clustering

# <center> Hitter </center>
<img src="https://www.ducksters.com/sports/baseball/hitter.jpg">

## Data importing

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
team_hit_2016 = pd.read_csv('team_hitter_scaling_2016.csv',index_col=0)
team_pit_2016 = pd.read_csv('team_pitcher_scaling_2016.csv',index_col=0)
team_hit_2016 = pd.merge(team_hit_2016,team_pit_2016[['G_ID','WLS','TB_SC']], on=['G_ID','TB_SC'])

Factor Analysis(FA)는 잠재변수를 찾는 방식이다. 잠재변수에 의해 X변수들이 결정된다고 가정한다. 순서는 다음과 같다.

1) **Choosing the Number of Factors**(Kaiser criterion & scree plot): <br>
고유값에 근거하여 적절한 요인의 개수를 판단 <br>
2) **Factor Analysis**: <br>
factor analysis를 통하여 factor가 어느 변수와 high factor loadings을 갖는지 살펴야함.

[참조](https://www.datacamp.com/community/tutorials/introduction-factor-analysis)

## Choosing the Number of Factors

In [4]:
features = ['RUN','P_HRA_RT','AVG','SLG','IsoP','OBP','OPS','RC','XR','wOBA','BABIP']
df = team_hit_2016.loc[:,features]
# Target y는 WLS로 설정한다.

NameError: name 'team_hit_2016' is not defined

In [ ]:
from factor_analyzer import FactorAnalyzer

fa = FactorAnalyzer()
fa.analyze(df, 10, rotation='varimax')
ev, v = fa.get_eigenvalues()
# ev

Kaiser Criterion에 의하여 고유값이 1보다 큰 경우만 factor로 취급한다. <br>
따라서 이 경우에는 factor의 갯수를 3개로 두자.

In [ ]:
plt.scatter(range(1, df.shape[1]+1),ev)
plt.plot(range(1,df.shape[1]+1),ev)
plt.grid(axis='y')
plt.show() #Factor가 3개일때만 1을 넘는다.

## Factor Analysis

In [ ]:
fa = FactorAnalyzer()
fa.analyze(df, 3, rotation="varimax")
#각 loading 값들이 최대한 서로 다르게 하기위해 Varimax rotation 실행.#fa.loadings
#fa.loadings
# fa.loadgins=L이며 L은 factor간 선형결합의 weight를 의미함. L 추정 중요
hit_loadings = fa.loadings.copy()
hit_loadings = hit_loadings.apply(lambda x: np.where(x==max(x)), axis=1)
hit_loadings = hit_loadings.apply(lambda x: int(sum(x)))
hit_loadings = hit_loadings.to_dict()
hit_fac = {n:[k for k in hit_loadings.keys() if hit_loadings[k] == n] for n in set(hit_loadings.values())}
for i in range(len(hit_fac.keys())):
    hit_fac['Factor'+str(i+1)] = hit_fac.pop(i)
hit_fac

**F1(타율,출루계열)** : AVG, OBP, wOBA, BABIP <br>
AVG(타율), OBP(출루율), wOBA(가중 출루율), BABIP

**F2(득점계열)** : RUN, P_HRA_RT, RC, XR <br>
RUN(득점), P_HHA_RT(득점권타율), RC(득점기여도), XR(득점 공헌도)

**F3(장타율계열)** : SLG, IsoP, OPS <br>
SLG(장타율), IsoP(순장타율), ops(출루율+장타율)

In [ ]:
fa.get_factor_variance()
# Total 86% cumulative Variance explained by the 3 factors.

In [ ]:
X = df.values
fa = FactorAnalysis(n_components=3)
X_fa = fa.fit_transform(X) # FA를 위해 E(X-mu)=0를 만들어주려고 mu를 빼주는듯?
df_fa =pd.DataFrame(data=X_fa, columns=['hit_f1','hit_f2','hit_f3'])
names = ['WLS','G_ID','T_ID','TB_SC']
for name in names:
    df_fa[name] = team_hit_2016[name]

In [ ]:
sns.pairplot(data=df_fa, hue='WLS')

In [ ]:
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans

In [ ]:
# Seaching for an optimal k (the elbow point)
sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_fa[['hit_f1','hit_f2','hit_f3']])
    sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5).fit(df_fa[['hit_f1','hit_f2','hit_f3']])
df_fa['cluster_hit'] = kmeans.labels_

In [ ]:
pd.crosstab(df_fa['cluster_hit'],df_fa['T_ID'])

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.axes(projection='3d')
ax.scatter3D(df_fa['hit_f1'],df_fa['hit_f2'],df_fa['hit_f3'],c=df_fa['cluster_hit'])
ax.set_xlabel('F1')
ax.set_ylabel('F2')
ax.set_zlabel('F3',rotation=90)
plt.show()

# <center> Pitcher </center>
<img src="http://sportsgroundproduction.blob.core.windows.net/cms/14732/newsarticles/63254_wo.jpg" width="220">

In [ ]:
team_pit_2016.head()

In [ ]:
features = ['R','ER','P_WHIP_RT','K/9','BB/9','K/BB','OAVG','OOBP','OSLG','OOPS','WHIP','BABIP','DICE',
                                               'ERA','RA9','FIP','kFIP','HR/9','H/9']
df = team_pit_2016.loc[:,features]

In [ ]:
fa = FactorAnalyzer()
fa.analyze(df, 10, rotation='varimax')
ev, v = fa.get_eigenvalues()
# ev

In [ ]:
plt.scatter(range(1, df.shape[1]+1), ev)
plt.plot(range(1, df.shape[1]+1),ev)
plt.grid(axis='y')
plt.show # 4개 선택

In [ ]:
fa = FactorAnalyzer()
fa.analyze(df, 4, rotation='varimax')
#fa.loadings
pit_loadings = fa.loadings.copy()
pit_loadings = pit_loadings.apply(lambda x: np.where(x==max(x)), axis=1)
pit_loadings = pit_loadings.apply(lambda x: int(sum(x)))
pit_loadings = pit_loadings.to_dict()

In [ ]:
pit_fac = {n:[k for k in pit_loadings.keys() if pit_loadings[k] == n] for n in set(pit_loadings.values())}
for i in range(len(pit_fac.keys())):
    pit_fac['Factor'+str(i+1)] = pit_fac.pop(i)
pit_fac

In [ ]:
fa.get_factor_variance()
# Total 87% cumulative Variance explained by the 4 factors.

In [ ]:
X = df.values
fa = FactorAnalysis(n_components=4)
X_fa = fa.fit_transform(X) # FA를 위해 E(X-mu)=0를 만들어주려고 mu를 빼주는듯?
df2_fa =pd.DataFrame(data=X_fa, columns=['pit_f1','pit_f2','pit_f3','pit_f4'])
for name in names:
    df2_fa[name] = team_pit_2016[name]

In [ ]:
# Seaching for an optimal k (the elbow point)
sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df2_fa[['pit_f1','pit_f2','pit_f3','pit_f4']])
    sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3).fit(df2_fa[['pit_f1','pit_f2','pit_f3','pit_f4']])
df2_fa['cluster_pit'] = kmeans.labels_

In [ ]:
sns.pairplot(data=df2_fa, hue='cluster_pit')

## K-means clustering

In [ ]:
final_fa = pd.merge(df_fa,df2_fa, on=['G_ID','T_ID','TB_SC','WLS'])

In [ ]:
kmeans = KMeans(n_clusters=3).fit(final_fa[['hit_f1','hit_f2','hit_f3','pit_f1','pit_f2','pit_f3','pit_f4']])
final_fa['cluster_play'] = kmeans.labels_

In [ ]:
cross_play = pd.crosstab(final_fa['cluster_play'],final_fa['T_ID'])
cross_play

In [ ]:
cross_hit = pd.crosstab(final_fa['cluster_hit'],final_fa['T_ID'])
cross_pit = pd.crosstab(final_fa['cluster_pit'],final_fa['T_ID'])

def cluster_max(crosstable,name):
    cluster_team = crosstable.apply(lambda x: np.where(x==max(x)))
    cluster_team = cluster_team.apply(lambda x: int(sum(x)))
    cluster_team = pd.DataFrame(cluster_team)
    cluster_team = cluster_team.reset_index()
    cluster_team.columns = ['T_ID',name]
    return(cluster_team)

In [ ]:
cluster = pd.merge(final_fa, cluster_max(cross_hit,'cluster_team_hit'),on='T_ID')
cluster = pd.merge(cluster, cluster_max(cross_pit,'cluster_team_pit'),on='T_ID')

In [ ]:
cluster = cluster[['G_ID','T_ID','TB_SC','WLS','cluster_team_hit','cluster_team_pit','hit_f1','hit_f2','hit_f3','pit_f1','pit_f2','pit_f3','pit_f4']]

In [ ]:
cluster.to_csv('cluster.csv', index=False)

In [ ]:
data = cluster.copy()
data = data.sort_values(['G_ID']).reset_index(drop=True)
data['T'] = None
data['B'] = None

In [ ]:
for i in range(data.shape[0]):
    data.iloc[i, 13] = data.iloc[i, 0][8:10]
    data.iloc[i, 14] = data.iloc[i, 0][10:12]

data = pd.concat([data, pd.get_dummies(data['T'], prefix='T', drop_first=True)], axis=1)
data = pd.concat([data, pd.get_dummies(data['B'], prefix='B', drop_first=True)], axis=1)
data = pd.concat([data, pd.get_dummies(data['TB_SC'], drop_first=True)], axis=1)
data = pd.concat([data, pd.get_dummies(data['cluster_team_hit'], prefix='HIT', drop_first=True)], axis=1)
data = pd.concat([data, pd.get_dummies(data['cluster_team_pit'], prefix='PIT', drop_first=True)], axis=1)

In [ ]:
data = data.drop(['G_ID', 'T_ID', 'TB_SC', 'cluster_team_hit', 'cluster_team_pit', 'T', 'B',
                 'hit_f1','hit_f2','hit_f3','pit_f1','pit_f2','pit_f3','pit_f4'], axis=1)
data.head()

X_train = data.iloc[:1250, 1:]
y_train = data.iloc[:1250, 0]
X_test = data.iloc[1250:, 1:]
y_test = data.iloc[1250:, 0]

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('예측 정확도:', round(sum(y_pred == y_test)/len(y_pred == y_test), 3))

In [ ]:
for k1 in range(3,8):
    for k2 in range(3,8):
        kmeans = KMeans(n_clusters=k1).fit(df_fa[['hit_f1','hit_f2','hit_f3']])
        df_fa['cluster_hit'] = kmeans.labels_

        kmeans = KMeans(n_clusters=k2).fit(df2_fa[['pit_f1','pit_f2','pit_f3','pit_f4']])
        df2_fa['cluster_pit'] = kmeans.labels_

        final_fa = pd.merge(df_fa,df2_fa, on=['G_ID','T_ID','TB_SC','WLS'])
        
        cross_hit = pd.crosstab(final_fa['cluster_hit'],final_fa['T_ID'])
        cross_pit = pd.crosstab(final_fa['cluster_pit'],final_fa['T_ID'])
        cluster = pd.merge(final_fa, cluster_max(cross_hit,'cluster_team_hit'),on='T_ID')
        cluster = pd.merge(cluster, cluster_max(cross_pit,'cluster_team_pit'),on='T_ID')

        data = cluster[['G_ID','T_ID','TB_SC','WLS','cluster_team_hit','cluster_team_pit','hit_f1','hit_f2','hit_f3','pit_f1','pit_f2','pit_f3','pit_f4']]
        data = data.sort_values(['G_ID']).reset_index(drop=True)
        data['T'] = None
        data['B'] = None
        for i in range(data.shape[0]):
            data.iloc[i, 13] = data.iloc[i, 0][8:10]
            data.iloc[i, 14] = data.iloc[i, 0][10:12]

        data = pd.concat([data, pd.get_dummies(data['T'], prefix='T', drop_first=True)], axis=1)
        data = pd.concat([data, pd.get_dummies(data['B'], prefix='B', drop_first=True)], axis=1)
        data = pd.concat([data, pd.get_dummies(data['TB_SC'], drop_first=True)], axis=1)
        data = pd.concat([data, pd.get_dummies(data['cluster_team_hit'], prefix='HIT', drop_first=True)], axis=1)
        data = pd.concat([data, pd.get_dummies(data['cluster_team_pit'], prefix='PIT', drop_first=True)], axis=1)

        data = data.drop(['G_ID', 'T_ID', 'TB_SC', 'cluster_team_hit', 'cluster_team_pit', 'T', 'B',
                 'hit_f1','hit_f2','hit_f3','pit_f1','pit_f2','pit_f3','pit_f4'], axis=1)

        X_train = data.iloc[:1250, 1:]
        y_train = data.iloc[:1250, 0]
        X_test = data.iloc[1250:, 1:]
        y_test = data.iloc[1250:, 0]

        model = RandomForestClassifier(random_state=0)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print('예측 정확도:',k1,k2,round(sum(y_pred == y_test)/len(y_pred == y_test), 3))

In [ ]:
cross_hit.apply(lambda x: np.where(x==max(x)))